#  Convolutional Neural Network (CNN, or ConvNet)

### Convolutional:
<img src='../../img/1_CnNorCR4Zdq7pVchdsRGyw.png'>
<img src='../../img/giadascxvfdgephy.gif'>

### add padding:
<img src='../../img/1 nYf_cUIHFEWU1JXGwnz-Ig.gif'>

### polling:
<img src='../../img/maxpool_animation.gif'>



In [35]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import RandomOverSampler
import random
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import IsolationForest